In [155]:
import numpy as np
import pandas as pd
import json
import difflib
from fuzzywuzzy import fuzz
pd.options.display.max_rows = 100

In [236]:
transfers_england_summer = pd.read_json('../Transfer_data/England_summer_2013.json')
transfers_england_winter = pd.read_json('../Transfer_data/England_winter_2012_13.json')

In [237]:
transfers_england_summer['value'] = transfers_england_summer['value'].astype(str)
transfers_england_winter['value'] = transfers_england_winter['value'].astype(str)

In [238]:
#Reading in the feature data
features12 = pd.read_pickle('../features12-13.pkl')
tr_features12 = features12.transpose()

In [239]:
#Concatenating all transfers for the season 12-13 for England
df_eng_12_13 = pd.concat([transfers_england_summer,transfers_england_winter],axis=0)

In [231]:
def fix_name(name_list):
    if len(name_list) == 1:
        return name_list[0]
    elif len(name_list) == 2:
        val = ' '.join([name_list[1],name_list[0]])
        return val.lstrip().rstrip()
    else:
        return 'Name Error'
##This function reverses the names which have commas and sets name column as index
def fix_names(df):
    df['name'] = df['name'].apply(lambda x:x.split(','))
    df['name'] = df['name'].apply(lambda x:fix_name(x))
    df['name'] = df['name'].astype(unicode)
    df = df.set_index('name')
    return df

In [234]:
check_list = ['Loan','Undisclosed','Swap','Free','Loan return']
#test_df = df_eng_12_13[~df_eng_12_13['value'].isin(check_list)]

In [241]:
##Testing the amount of transfer money we can account for after name fix
sum(trial_df[~trial_df['value'].isin(check_list)]['value'].astype(float))

435850000.0

##Function to clean and match names and join the values

In [224]:
def join_df(transfer_df,stats_df):
    test_df = transfer_df
    test_df['check_name'] = test_df.index.map(lambda x: difflib.get_close_matches(x, stats_df.index,1))
    test_df = test_df[test_df['check_name'].str.len() != 0]
    test_df['check_name'] = test_df['check_name'].apply(lambda x:x[0])
    test_df['names'] = test_df.index
    test_df['sim_score'] = test_df[['check_name','names']].apply(lambda x:fuzz.ratio(x[0],x[1]),axis=1)
    to_be_joined = test_df[test_df['sim_score'] > 82]
    to_be_joined['ref_name'] = to_be_joined['check_name']
    to_be_joined = to_be_joined.set_index('check_name')
    trial_df = stats_df.join(to_be_joined,how='inner')
    return trial_df

####See trial df for the data

In [ ]:
trial_df = join_df(fix_names(df_eng_12_13),tr_features12)

In [226]:
#Loading data for Spain for 2012-13
transfers_spain_summer = pd.read_json('../Transfer_data/Spain_summer_2013.json')
transfers_spain_winter = pd.read_json('../Transfer_data/Spain_winter_2012_13.json')

In [227]:
df_spn_12_13 = pd.concat([transfers_spain_summer,transfers_spain_winter],axis=0)